In [1]:
"""
Hypothesis
The distribution is quite diverse, following the flat power law
"""
# step 1: get distribution for a single prediction

'\nHypothesis\nThe distribution is quite diverse, following the flat power law\n'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from datasets import load_dataset
import torch

In [3]:
cc_news = load_dataset('cc_news')['train']

from random import randint
APPROX_LETTERS_PER_TOKEN = 7

def sample_sequences(tokenizer, dataset, ctx_size, num_samples):
    max_attempts = 10_000
    tokens = []
    for _ in range(num_samples):
        for _ in range(max_attempts):
            i = randint(0, len(dataset))
            text = dataset[i]['text']
            if len(text) < ctx_size * APPROX_LETTERS_PER_TOKEN:
                continue
            else:
                tokenized = tokenizer(text[:ctx_size*APPROX_LETTERS_PER_TOKEN], return_tensors='pt').input_ids
                tokens.append(tokenized[0, :ctx_size])
                break
    return tokens



Found cached dataset cc_news (/app/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/e3d5612f02fe5f11826a0d9614328b1772e27e5d685f4ec438e7f768e4581734)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-hf")
model = model.cuda()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
sequences = sample_sequences(tokenizer, cc_news, 10, 100)

In [6]:
query = sequences[0][None, :].cuda()
# res = model.generate(query, max_new_tokens=20, output_scores=True, do_sample=True, num_return_sequences=3)

In [7]:
# type(model)
res = model(query)


In [8]:
res.logits.shape

torch.Size([1, 20, 32000])

In [24]:
idx = torch.topk(res.logits, k=2).indices

In [15]:
query

tensor([[    1, 21184,  1341,   265, 10120, 16369, 29901, 21184,  1341,   265,
         10120,   830,  4011,  3824,   751,  4254, 29871, 29906, 29900, 29896]],
       device='cuda:0')

In [25]:
[query[0][i+1] in idx[0][i] for i in range(len(query[0]) - 1)]

[False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True]

In [27]:
query

tensor([[    1, 21184,  1341,   265, 10120, 16369, 29901, 21184,  1341,   265,
         10120,   830,  4011,  3824,   751,  4254, 29871, 29906, 29900, 29896]],
       device='cuda:0')

21184 tensor([396, 917], device='cuda:0')
1341 tensor([29901, 29899], device='cuda:0')
265 tensor([ 265, 2463], device='cuda:0')
10120 tensor([10120,   357], device='cuda:0')
16369 tensor([16369,  9266], device='cuda:0')
29901 tensor([  313, 29901], device='cuda:0')
21184 tensor([21184,  3824], device='cuda:0')
1341 tensor([ 1341, 14438], device='cuda:0')
265 tensor([265, 856], device='cuda:0')
10120 tensor([10120,   357], device='cuda:0')
830 tensor([16369,  8081], device='cuda:0')
4011 tensor([ 4011, 14520], device='cuda:0')
3824 tensor([4231,  373], device='cuda:0')
751 tensor([  751, 28144], device='cuda:0')
4254 tensor([4254,  856], device='cuda:0')
29871 tensor([29871, 17212], device='cuda:0')
29906 tensor([29906, 29896], device='cuda:0')
29900 tensor([29900, 29896], device='cuda:0')
29896 tensor([29896, 29906], device='cuda:0')


In [33]:

res2 = model(sequences[0][None, :10].cuda())


In [38]:
res2.logits[0, 9, :10]

tensor([-5.1044, -8.2341,  5.8011, -0.0318, -1.4130,  2.4803, -1.1917, -1.9396,
         0.2337, -1.7546], device='cuda:0', grad_fn=<SliceBackward0>)

In [39]:
res.logits[0, 9, :10]

tensor([-5.1044, -8.2341,  5.8011, -0.0318, -1.4130,  2.4803, -1.1917, -1.9396,
         0.2337, -1.7546], device='cuda:0', grad_fn=<SliceBackward0>)

In [45]:
for seq in sequences[:3]:
    res = model.generate(seq[None, :].cuda(), max_new_tokens = 100)
    # model.generate()

In [48]:
out = model(res.cuda())

In [61]:
out.past_key_values[3][0].shape

torch.Size([1, 32, 110, 128])

In [62]:
pkv = out.past_key_values

In [72]:
#32x2 x 1x32x110x128
pkv[0][1].shape

torch.Size([1, 32, 110, 128])

In [73]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [87]:
pkv[28][1].shape

torch.Size([1, 32, 110, 128])

In [105]:
logits = out.logits
log_n = logits[0] / torch.norm(logits[0], dim=-1, keepdim=True)


In [119]:
log_probs = torch.log(torch.softmax(logits[0], dim=-1))


In [123]:
s

torch.return_types.topk(
values=tensor([[-2.5894, -2.6135, -3.1171,  ..., -5.3640, -5.3865, -5.4500],
        [-1.7240, -2.5850, -3.1739,  ..., -4.7889, -4.8943, -4.9039],
        [-2.1116, -2.3623, -2.4095,  ..., -5.1073, -5.3166, -5.4554],
        ...,
        [-1.4146, -1.8259, -2.2028,  ..., -5.3303, -5.4250, -5.4640],
        [-0.2075, -1.9197, -3.7225,  ..., -9.7803, -9.8120, -9.8671],
        [-0.9454, -0.9603, -2.4114,  ..., -7.8228, -8.6324, -8.6560]],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([[  396,   917,   450,  ...,   349,   317,  2648],
        [ 1486,   852,   272,  ..., 29965, 29915, 29903],
        [ 2813,   728,   886,  ...,   328, 22617, 29899],
        ...,
        [ 1207,  1831,  5839,  ...,  2186, 28269,  4808],
        [  263,   967,   278,  ...,   322,   670,   701],
        [ 2186, 10608,  9262,  ..., 15047,  9226,  1246]], device='cuda:0'))